# Transfer Learning for NLP using BERT

In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt

import torch 
from torch import nn, optim 
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torchmetrics.functional import accuracy

import pytorch_lightning as pl 
from pytorch_lightning.callbacks import TQDMProgressBar

import transformers 
from transformers import BertModel, BertConfig, AutoModel, BertTokenizerFast 

## Load the dataset

In [ ]:
pub_health_train = pd.read_csv('../data/PUBHEALTH/train.tsv', sep='\t')
pub_health_test = pd.read_csv('../data/PUBHEALTH/test.tsv', sep='\t')

In [ ]:
pub_health_train.head()

In [ ]:
pub_health_train = pub_health_train[pub_health_train['label'] != 'snopes']
pub_health_train = pub_health_train[['main_text', 'label']]

# Drop the rows where at least one element is missing.
pub_health_train = pub_health_train.dropna(subset=['main_text', 'label'])
pub_health_train.head()

In [ ]:
pub_health_test = pub_health_test[['main_text', 'label']]
pub_health_test = pub_health_test.dropna(subset=['main_text', 'label'])
pub_health_test.head()

In [ ]:
pub_health_train['label'] = pub_health_train['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)
pub_health_test['label'] = pub_health_test['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)

In [ ]:
class HealthClaimClassifier(pl.LightningModule):

    def __init__(self, max_seq_len=512, batch_size=128, learning_rate=1e-3):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.loss = nn.CrossEntropyLoss()

        self.pretrain_model = AutoModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.pretrain_model.eval() 
        for param in self.pretrain_model.parameters():
            param.requires_grad = False
        
        self.new_layers = nn.Sequential(
            nn.Linear(768, 512), 
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(512, 4),
            nn.LogSoftmax(dim=1),
        )
    
    def prepare_data(self):
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', return_dict=False)

        tokens_train = tokenizer.batch_encode_plus(
            pub_health_train['main_text'].tolist(), 
            max_length=self.max_seq_len, 
            pad_to_max_length=True, 
            truncation=True, 
            return_token_type_ids=False
        )

        tokens_test = tokenizer.batch_encode_plus(
            pub_health_test['main_text'].tolist(),
            max_length=self.max_seq_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=False
        )

        self.train_seq = torch.tensor(tokens_train['input_ids'])
        self.train_mask = torch.tensor(tokens_train['attention_mask'])
        self.train_y = torch.tensor(pub_health_train['label'].tolist())

        self.test_seq = torch.tensor(tokens_test['input_ids'])
        self.test_mask = torch.tensor(tokens_test['attention_mask'])
        self.test_y = torch.tensor(pub_health_test['label'].tolist())
    
    def forward(self, seq, mask):
        _, output = self.pretrain_model(seq, attention_mask=mask)
        output = self.new_layers(output)
        return output
    
    def train_dataloader(self):
        train_dataset = TensorDataset(self.train_seq, self.train_mask, self.train_y)
        self.train_dataloader_obj = DataLoader(train_dataset, batch_size=self.batch_size)
        return self.train_dataloader_obj
    
    def test_dataloader(self):
        test_dataset = TensorDataset(self.test_seq, self.test_mask, self.test_y)
        self.test_dataloader_obj = DataLoader(test_dataset, batch_size=self.batch_size)
        return self.test_dataloader_obj
    
    def training_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self(seq, mask)
        preds = torch.argmax(outputs, dim=1)

        train_accuracy = accuracy(preds, targets)
        loss = self.loss(outputs, targets)

        self.log('train_accuracy', train_accuracy, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return {'loss': loss, 'train_accuracy': train_accuracy}
    
    def test_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self.forward(seq, mask)
        preds = torch.argmax(outputs, dim=1)

        test_accuracy = accuracy(preds, targets)
        loss = self.loss(outputs, targets)
        return {'loss': loss, 'test_accuracy': test_accuracy}
    
    def test_epoch_end(self, outputs):
        test_outs = []
        for test_out in outputs:
            out = test_out['test_accuracy']
            test_outs.append(out)
        total_test_accuracy = torch.stack(test_outs).mean()
        self.log('total_test_accuracy', total_test_accuracy, on_step=False, on_epoch=True)
        return total_test_accuracy
    
    def configure_optimizers(self):
        params = self.parameters()
        optimizer = torch.optim.Adam(params=params, lr=self.learning_rate)
        return optimizer

In [ ]:
model = HealthClaimClassifier()
trainer = pl.Trainer(fast_dev_run=True, accelerator='gpu', devices=1)
trainer.fit(model)

In [ ]:
model = HealthClaimClassifier()
bar = TQDMProgressBar()
trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10, accelerator='gpu', devices=1, callbacks=[bar])
trainer.fit(model)

In [ ]:
trainer.test()